In [84]:
import pandas as pd
import numpy as np
from scipy.spatial import distance, distance_matrix
from scipy.spatial.distance import pdist
from pandas import ExcelWriter
from pandas import ExcelFile

In [85]:
X_8 = pd.read_excel('8clusters.xlsx', sheetname='Data')
X_12 = pd.read_excel('12clusters.xlsx', sheetname='Data')
X_14 = pd.read_excel('14clusters.xlsx', sheetname='Data')
datasets = [X_8, X_12, X_14]

In [86]:
def centrs_cluster(X):
    data = X.iloc[:,2:]
    return [data.loc[X.iloc[:,1] == i].mean().as_matrix() 
            for i in sorted(X.iloc[:,1].unique())]

def differences(X, m, metric):
    return [metric(u,m) for index, u in X.iterrows()]

def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)
    
def up(X, metric):
    clusters = X.iloc[:,1].value_counts().index.tolist()
    num_objects = X.iloc[:,1].value_counts()
    centres = centrs_cluster(X)
    data = X.iloc[:,2:]
    return np.mean([ sum(
                     differences(data.loc[X.iloc[:,1] == c], centres[c-1], metric)
                     ) / num_objects[c] 
                        for c in clusters ])

def down(X, metric="euclidean"):
    clusters = X.iloc[:,1].value_counts().index.tolist()
    centres = centrs_cluster(X)
    return sum(pdist(centrs_cluster(X), metric=metric))/nCr(len(clusters), 2)

In [87]:
inside_group = np.array([up(X, distance.euclidean) for X in datasets])
outside_group = np.array([down(X) for X in datasets])
devided = inside_group/outside_group

In [88]:
pd.DataFrame([["inside", *inside_group], 
              ["outside", *outside_group], 
              ["devided", *devided]], 
              columns=["name", "8 cluster", "12 cluster", "14 cluster"])

,name,8 cluster,12 cluster,14 cluster
0,inside,36.842863,28.537577,31.603595
1,outside,66.920037,76.606897,77.296405
2,devided,0.550551,0.372520,0.408862


In [89]:
inside_group = np.array([up(X, distance.sqeuclidean) for X in datasets])
outside_group = np.array([down(X, "sqeuclidean") for X in datasets])
devided = inside_group/outside_group

In [90]:
pd.DataFrame([["inside", *inside_group], 
              ["outside", *outside_group], 
              ["devided", *devided]], 
              columns=["name", "8 cluster", "12 cluster", "14 cluster"])

,name,8 cluster,12 cluster,14 cluster
0,inside,1542.635191,1093.817289,1202.456790
1,outside,4964.938492,6959.309481,7025.475336
2,devided,0.310706,0.157173,0.171157
